## 环境测试

In [ ]:
# 测试
print("hello")

**安装LangChain**

In [ ]:
!pip install --upgrade langchain -i https://pypi.org/simple

In [ ]:
!pip install --upgrade langchain -i https://pypi.org/simple

In [ ]:
# 安装openapi
!pip install openai==v0.28.1 -i https://pypi.org/simple

**引入openaikey**

**引入openai**

In [43]:
!pip3 install langchain

UsageError: Line magic function `%pip3` not found.


In [42]:
!/opt/miniconda3/bin/python -m pip install openai  

UsageError: Line magic function `%/opt/miniconda3/bin/python` not found.


In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

## 阿里千问api

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)
client = OpenAI(
    api_key=os.getenv("ALI_QW_API_KEY"),  
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    model="qwen-long",
    messages=[
        {
            'role': 'system',
            'content': 'You are a helpful assistant.'
        },
        {
            'role': 'system',
            'content': '大型语言模型(llm)已经彻底改变了人工智能领域，使以前被认为是人类独有的自然语言处理任务成为可能...'
        },
        {
            'role': 'user',
            'content': '文章讲了什么？'
        }
    ],
    stream=True
)
for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].dict())


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv(override=True)

template = """Question: {question}
Let's think step by step.
Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

# 修改ChatOpenAI的配置以使用阿里千问API
llm = ChatOpenAI(
    model_name="qwen-long",
    openai_api_key=os.getenv("ALI_QW_API_KEY"),
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

llm_chain = LLMChain(prompt=prompt, llm=llm)
question = """What is the population of the capital of the country
where the Olympic Games were held in 2008?"""
llm_chain.run(question)

"Sure, let's break this down step by step.\n\n1. **Identify the year and location of the Olympic Games**: \n   - The Olympic Games were held in 2008.\n   - In 2008, the Summer Olympics (also known as the Games of the XXIX Olympiad) were hosted by Beijing, China.\n\n2. **Determine the capital city**: \n   - Beijing is the capital city of China.\n\n3. **Find the current population of Beijing**:\n   - As of 2021, the estimated population of Beijing was approximately 21.54 million people.\n\nTherefore, the population of the capital of the country where the Olympic Games were held in 2008 (Beijing, China) is around 21.54 million. Note that the exact figure may vary slightly depending on the source and the specific year of the estimate."

In [12]:
# 查看python执行目录
import sys
print(sys.executable)

/opt/miniconda3/bin/python


# 使用工具

In [44]:
%pip install wikipedia
%pip install llm-math

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement llm-math (from versions: none)
ERROR: No matching distribution found for llm-math
Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.agents import load_tools, initialize_agent, AgentType
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv(override=True)

template = """Question: {question}
Let's think step by step.
Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

# 修改ChatOpenAI的配置以使用阿里千问API
llm = ChatOpenAI(
    model_name="qwen-long",
    openai_api_key=os.getenv("ALI_QW_API_KEY"),
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1"
)
tools = load_tools(["wikipedia","llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

question = """What is the population of the capital of the country
where the Olympic Games were held in 2008?"""
agent.run(question)



> Entering new AgentExecutor chain...
I need to find out which country hosted the Olympic Games in 2008 and then find the population of its capital.
Action: wikipedia
Action Input: 2008 Summer Olympics
Observation: Page: 2008 Summer Olympics
Summary: The 2008 Summer Olympics (Chinese: 2008年夏季奥运会; pinyin: Èr Líng Líng Bā Nián Xiàjì Àoyùnhuì), officially the Games of the XXIX Olympiad (Chinese: 第二十九届夏季奥林匹克运动会; pinyin: Dì Èrshíjiǔ Jiè Xiàjì Àolínpǐkè Yùndònghuì) and officially branded as Beijing 2008 (Chinese: 北京2008; pinyin: Běijīng èr líng líng bā), were an international multisport event held from 8 to 24 August 2008, in Beijing, China. A total of 10,942 athletes from 204 National Olympic Committees (NOCs) competed in 28 sports and 302 events, one event more than those scheduled for the 2004 Summer Olympics. This was the first time China had hosted the Olympic Games, and the third time the Summer Olympic Games had been held in East Asia, following the 1964 Olympics in Tokyo, Japan, an

'The population of Beijing, the capital of China where the 2008 Olympic Games were held, is more than 22 million residents.'

# 使用ConversationChanin 记忆

In [16]:
from langchain import OpenAI, ConversationChain
from langchain import PromptTemplate, LLMChain
from langchain.agents import load_tools, initialize_agent, AgentType
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv(override=True)
chatbot_llm = llm = ChatOpenAI(
    model_name="qwen-long",
    openai_api_key=os.getenv("ALI_QW_API_KEY"),
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1"
)
chatbot = ConversationChain(llm=chatbot_llm , verbose=True)
chatbot.predict(input='Hello')

/opt/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello
AI:

> Finished chain.


"Hello there! How can I help you today? I'm full of information and ready to chat. What's on your mind?"

# 使用嵌入

In [41]:
%pip install pypdf
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [37]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
import os
from dotenv import load_dotenv
from typing import List
import dashscope

# 加载环境变量
load_dotenv(override=True)

# 设置阿里云API密钥
dashscope.api_key = os.getenv("ALI_QW_API_KEY")

class AliEmbeddings(Embeddings):
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        results = []
        for text in texts:
            response = dashscope.TextEmbedding.call(model="text-embedding-v3", input=text)
            if response.status_code == 200:
                results.append(response.output['embeddings'][0]["embedding"])
            else:
                raise Exception(f"Error: {response.code}, {response.message}")
        return results

    def embed_query(self, text: str) -> List[float]:
        response = dashscope.TextEmbedding.call(model="text-embedding-v3", input=text)
        if response.status_code == 200:
            return response.output['embeddings'][0]["embedding"]
        else:
            raise Exception(f"Error: {response.code}, {response.message}")

# 加载PDF文档
loader = PyPDFLoader("./云信小号接口文档.pdf")
pages = loader.load_and_split()

# 创建嵌入和向量存储
embeddings = AliEmbeddings()
db = FAISS.from_documents(pages, embeddings)

# 执行相似性搜索
q = "小号有几种模式绑定?"
result = db.similarity_search(q)[0]
print(result)

True

page_content='云呼平台接口文档 
7  
2.1.3  AXyb模式绑定  
1) 功能代码 : middle NumberAX  
2) 功能介绍  
提前绑定小号号码、 号码 A的关系， autoBindY 为1时，号码 B呼叫小号时自动绑定号码 Y，在号码 A上显
示号码 Y，autoBindY  为0时，需要用接口 middleNumberXYB 绑定号码 Y和B。 
3) 接口参数  
参数 参数说明  数据类型  必须 备注 
middleNumber  小号号码  String  是 平台分配的小号 X 
bindNumberA  绑定小号的号码 A String  是 格式要求：手机号  
callRec  是否对通话录音，  
1：开启， 0：关闭  int 否 默认开启，线路不支持时无效  
maxBindingTime  绑定关系有效时长  int 否 为空表示绑定关系永久，单位 :秒 
callbackUrl  呼叫结果推送地址，
http协议 String  否 http,可自由指定呼叫结果推送的地址  
mode  绑定模式  int 是 必须填 4: AXyb 模式 
autoBindY  号码 B呼入时是否自
动绑定号码 Y String  否 缺省为自动分配号码 Y 
4) 响应参数  
参数 参数说明  数据类型  备注 
result  响应结果  String  000000 表示提交成功，详见 响应代码  
message  响应代码具体原因  String  详见响应代码  
bindId  绑定 ID String  小号绑定关系唯一标识  
middleNumber  绑定的小号  String  绑定成功的小号号码  
 
 
2.1.4  XYB模式绑定  
7) 功能代码 : middleNumberXYB  
8) 功能介绍  
绑定小号 X、小号 Y，号码 B的关系。' metadata={'source': './云信小号接口文档.pdf', 'page': 8}


# 向量信息检索

In [38]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
import os
from dotenv import load_dotenv
from typing import List
import dashscope

# 加载环境变量
load_dotenv(override=True)

# 设置阿里云API密钥
dashscope.api_key = os.getenv("ALI_QW_API_KEY")

class AliEmbeddings(Embeddings):
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        results = []
        for text in texts:
            response = dashscope.TextEmbedding.call(model="text-embedding-v3", input=text)
            if response.status_code == 200:
                results.append(response.output['embeddings'][0]["embedding"])
            else:
                raise Exception(f"Error: {response.code}, {response.message}")
        return results

    def embed_query(self, text: str) -> List[float]:
        response = dashscope.TextEmbedding.call(model="text-embedding-v3", input=text)
        if response.status_code == 200:
            return response.output['embeddings'][0]["embedding"]
        else:
            raise Exception(f"Error: {response.code}, {response.message}")

# 加载PDF文档
loader = PyPDFLoader("./云信小号接口文档.pdf")
pages = loader.load_and_split()

# 创建嵌入和向量存储
embeddings = AliEmbeddings()
db = FAISS.from_documents(pages, embeddings)

# 执行相似性搜索
q = "小号有几种模式绑定?"
chain = RetrievalQA.from_llm(llm=llm, retriever=db.as_retriever())
chain(q, return_only_outputs=True)

True

/opt/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'result': '根据提供的信息，小号至少有两种模式绑定：AXyb模式和XYB模式。但是，也可能存在其他模式，如AX/AXN /AXx模式等。这些模式允许用户通过不同的方式进行小号绑定，以便满足不同的需求。具体的绑定方式和功能可能会有所不同，请参照具体的模式介绍。如果你需要更确切的数量，请提供更多的信息。'}

# 插件
开发GPT-4插件涉及创建一个API，使用OpenAPI规范来指定其行为，并在插件清单文件中描述插件及其用法。通过这种设置，GPT-4可以有效地充当智能的API调用者，从而获得超越文本生成的能力。